In [2]:
from pathlib import Path
import json

TEXT = Path("/Users/joregan/Playing/wolne-match/text")
WHISPER = Path("/Users/joregan/Playing/wolne-match/whisperx")

In [ ]:
!pip install mosestokenizer

In [7]:
test = TEXT / "zloczynca.txt"

textlines = []
with open(test, "r") as f:
    for line in f:
        line = line.strip()
        if line == "":
            continue
        textlines.append(line)

In [9]:
test2 = "/Users/joregan/Playing/wolne-match/whisperx/zloczynca_000.json"

def read_whisperx_json(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

In [18]:
from mosestokenizer import MosesSentenceSplitter

def get_sentences_poorly(text):
    outputs = []
    with MosesSentenceSplitter("pl") as splitter:
        for line in text:
            sentences = splitter([line])
            outputs.extend(sentences)
    return outputs

In [28]:
def rejoin_poorly_split_sentences(sentences):
    outputs = []
    i = 0
    while i < len(sentences) - 1:
        if sentences[i].endswith(":") and sentences[i + 1][0].islower():
            outputs.append(sentences[i] + " " + sentences[i + 1])
            i += 2
        else:
            outputs.append(sentences[i])
            i += 1
    return outputs

In [ ]:
import re

def continue_splitting(sentence):
    pattern = r'(?<=[.?!]) — (?=[A-ZĄĆĘŁŃÓŚŹŻ])'
    parts = re.split(pattern, sentence)
    return parts

In [61]:
sentence = "'— Nigdy nie kłamałem, a teraz od razu niby kłamię… — mruczy Denis, mrugając oczyma. — Czyż można bez ciężarka, panie sędzio?'"
continue_splitting(sentence)

["'— Nigdy nie kłamałem, a teraz od razu niby kłamię… — mruczy Denis, mrugając oczyma.",
 "Czyż można bez ciężarka, panie sędzio?'"]

In [83]:
rejoin_poorly_split_sentences(get_sentences_poorly(textlines))

stdbuf was not found; communication with perl may hang due to stdio buffering.


['Anton Czechow',
 'Złoczyńca',
 'Przed sędzią śledczym stoi mały, niezwykle chudy chłopek w zgrzebnej koszuli i łatanych portkach.',
 'Jego zarośnięta, dziobata twarz i oczy, ledwo widoczne spoza gęstych, obwisłych brwi, mają wyraz posępnej surowości.',
 'Gęsta, splątana, dawno nie czesana czupryna nadaje jeszcze więcej surowości jego twarzy.',
 '— Denis Grigoriew! — zaczyna sędzia — podejdź bliżej i odpowiadaj na moje pytania.',
 'Siódmego lipca dróżnik kolejowy Akinfow, obchodząc rano tor, schwytał cię na odkręcaniu muterki, którą przymocowuje się szynę do podkładu.',
 'Oto ta muterka.',
 'Z nią cię zatrzymał.',
 'Czy tak było?',
 '— Czego?',
 '— Czy tak było, jak mówi Akinfow?',
 '— Wiadoma rzecz, że było.',
 '— Dobrze, a po cóż odkręcał?',
 '— Czego?',
 '— Daj spokój z tym „czego”, a odpowiadaj na pytania: po coś odkręcał?',
 '— Gdyby nie była potrzebna, to bym nie odkręcał — odpowiada ochrypniętym głosem Denis, patrząc na sufit.',
 '— Na co ci była potrzebna muterka?',
 '— Muterk

In [ ]:
def clean_word(word):
    word = word.strip(",;:!?—…„”\"“.«»*()[]‘/\\")
    if word:
        return word.lower()
    else:
        return ""

def clean_text(text):
    words = text.split(" ")
    cleaned_words = []
    for word in words:
        word = word.strip(",;:!?—…„”\"“.«»*()[]‘/\\")
        if word:
            cleaned_words.append(word.lower())
    return " ".join(cleaned_words)

In [66]:
clean_text("Jego zarośnięta, dziobata twarz i oczy, ledwo widoczne spoza gęstych, obwisłych brwi, mają wyraz posępnej surowości.")

'jego zarośnięta dziobata twarz i oczy ledwo widoczne spoza gęstych obwisłych brwi mają wyraz posępnej surowości'

In [ ]:
for segment in read_whisperx_json(test2)["segments"]:
    cleaned = clean_text(segment["text"].strip(" "))
    print(cleaned)

In [70]:
def read_book(book: str):
    paragraphs = []
    with open(book) as f:
        for line in f.readlines():
            line = line.strip()
            if line == "":
                continue
            if line == "-----":
                break
            paragraphs.append(line)
    return paragraphs

In [72]:
def read_specific_norms(norm_file):
    corrections = {}
    normalisations = {}
    with open(norm_file) as f:
        for line in f.readlines():
            line = line.strip()
            if not "\t" in line:
                continue
            parts = line.split("\t")
            name = parts[0]
            if len(parts) == 4 and parts[3] == "C":
                if not name in corrections:
                    corrections[name] = {}
                corrections[name][parts[1]] = parts[2]
            elif len(parts) == 3:
                if not name in normalisations:
                    normalisations[name] = {}
                normalisations[name][parts[1]] = parts[2]
    return corrections, normalisations

In [73]:
cor, norm = read_specific_norms("/Users/joregan/Playing/wolnelektury-speech-corpus/specific-norms.tsv")

In [82]:
def read_lines(file: str):
    return [x for x in enumerate(read_book(file), start=1)]

In [ ]:
def get_sentences(lines):
    for para_id, para in lines:
        sentences = get_sentences_poorly(para)
        for sentence in sentences:
            yield paraid, sentence